In [1]:
# once i have wholegenome_offset.trees
import pandas as pd
import tskit
import allel
import random
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import tsinfer
import pyslim
import json
import os

In [3]:
def overlap_neutral_mut (ts_new, ts, mapper_realid_metadataid):
    ## extract surviving ndoes and comapre them to our old ndoes to place mtuations in the right place
    surviving_nodes = []
    for i in ts_new.tables.nodes:
        surviving_nodes.append(i.metadata['slim_id'])
    ## new nodes id and the ids i gave them in the past
    new_mapper = pd.DataFrame({'new_ids': range(0, len(ts_new.tables.nodes)), 'my_ids_metadata':surviving_nodes})
    ## map old nodes with new nodes
    mapper_lost_nodes = new_mapper.merge(mapper_realid_metadataid, left_on = 'my_ids_metadata', right_on = 'my_ids_metadata', how= 'right')

    ## create a mask to only keep from the old nodes the ones that survived the simulation
    mask = mapper_lost_nodes['new_ids'].notna()

    tables_og = ts.dump_tables()

    ## now filter old tables only based on surviving nodes 
    tables_og.nodes.replace_with(tables_og.nodes[mask])

    ## now filter mutation table based on the surviving nodes, for that, extract the nodes 
    old_nodes = tables_og.mutations.node

    old_nodes = pd.Series(old_nodes)

    old_nodes.name = 'old_nodes'

    ## create a dataframe relating the new and old nodes
    replace_oldbynew_nodes = pd.merge(old_nodes, mapper_lost_nodes, left_on ='old_nodes', right_on = 'real_id', how= 'left')

    ## create a mask to filter out all the mutations than has been lost 
    mask_mutations_lost = replace_oldbynew_nodes['new_ids'].notna()

    ## filter out mutations that has been lost 
    table_mutations = tables_og.mutations[mask_mutations_lost]

    ## now replace the old nodes ids by the new nodes ids with the mapper
    ids_to_replace = replace_oldbynew_nodes.dropna()['new_ids']
    table_mutations.node = np.array(ids_to_replace.astype('int32'))

    ## and jsut set the sites from 0 to the length of mutation table 
    table_mutations.site = np.array(range(0, len(table_mutations))).astype('int32')

    ## apply the same filter from the mutations table to the sites table 
    table_sites = tables_og.sites[mask_mutations_lost]  

    ## now replace all this filter old tables in the new tree seq! 
    new_tables = ts_new.dump_tables()

    new_tables.mutations.replace_with(table_mutations)

    new_tables.sites.replace_with(table_sites)

    ## make sure to compure mutations parents
    new_tables.compute_mutation_parents()

    ## create tree seq based on tables
    tree_nm = new_tables.tree_sequence()

    return tree_nm.simplify()

In [4]:
def convert_tree_to_vcf (tree,name_vcf):
    # create a vcf file from the treeseq 
    with open(name_vcf, 'w') as file:
        # Pass the file object as the output parameter
        tree.write_vcf(output=file)

In [5]:
#import the old tree
ts_old = tskit.load("../../treeseq/wholegenome_offset_baselinetree.trees")
#import mapper old nodes to new nodes
mapper_realid_metadataid = pd.read_csv('../../treeseq/mapper_realid_metadataid_wholegenome.csv')

In [27]:
selfing = tskit.load('drift_nocrossin_norec1_result.trees')

ts_nm = overlap_neutral_mut(selfing, ts_old, mapper_realid_metadataid)

convert_tree_to_vcf(ts_nm, f'selfing.vcf')

In [ ]:
selfing

In [2]:
vcf_drift_norecomb = allel.read_vcf(f'selfing.vcf')

In [3]:
len(vcf_drift_norecomb['samples'])

900

In [4]:
geno_drift_no_recomb = vcf_drift_norecomb["calldata/GT"]

In [5]:
alt_al_per_pos_nr = geno_drift_no_recomb.sum(axis=2) 

In [6]:
alt_al_per_pos_nr = np.swapaxes(alt_al_per_pos_nr, 0, 1)

In [7]:
## count ecotypes 
genotpye_counts = {}
for i in alt_al_per_pos_nr:
    sample = i.tobytes()
    if sample in genotpye_counts:
        genotpye_counts[sample] += 1
    else:
        genotpye_counts[sample] = 1

In [8]:
#3hundred something 

In [9]:
len(genotpye_counts.values())

333

In [11]:
genotpye_counts.values()

dict_values([3, 3, 3, 7, 4, 3, 2, 6, 5, 5, 1, 4, 4, 4, 6, 3, 1, 1, 7, 1, 3, 4, 5, 3, 3, 4, 2, 3, 1, 5, 7, 5, 1, 4, 3, 2, 6, 3, 6, 6, 1, 7, 4, 2, 2, 3, 1, 3, 3, 3, 2, 5, 4, 3, 5, 3, 2, 7, 1, 5, 11, 3, 4, 1, 2, 1, 2, 1, 5, 3, 1, 3, 4, 8, 8, 3, 2, 5, 2, 5, 2, 4, 1, 7, 1, 6, 3, 3, 3, 1, 3, 8, 2, 2, 1, 4, 4, 1, 4, 3, 6, 1, 3, 2, 4, 2, 1, 3, 2, 5, 1, 7, 3, 2, 4, 3, 1, 3, 7, 3, 1, 2, 8, 2, 1, 1, 4, 1, 1, 10, 1, 2, 3, 2, 7, 2, 2, 6, 4, 1, 1, 1, 3, 2, 1, 4, 2, 4, 4, 1, 3, 2, 4, 5, 2, 6, 1, 1, 9, 3, 1, 6, 1, 2, 3, 5, 1, 7, 1, 2, 3, 1, 2, 2, 2, 7, 4, 3, 4, 2, 3, 5, 7, 4, 3, 5, 2, 2, 5, 1, 1, 2, 3, 6, 1, 2, 4, 3, 1, 2, 2, 1, 1, 2, 1, 3, 3, 4, 1, 1, 1, 4, 1, 4, 1, 2, 2, 1, 1, 3, 2, 4, 7, 1, 6, 3, 1, 2, 1, 3, 4, 5, 3, 2, 2, 1, 2, 2, 2, 1, 1, 4, 1, 1, 1, 2, 2, 2, 2, 5, 2, 2, 2, 3, 2, 3, 2, 2, 1, 1, 1, 1, 2, 1, 1, 3, 1, 2, 2, 3, 1, 1, 1, 1, 1, 2, 2, 1, 1, 1, 1, 3, 1, 1, 1, 5, 1, 4, 1, 1, 3, 1, 4, 3, 3, 1, 2, 2, 1, 2, 2, 1, 1, 1, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 3, 1, 1, 1, 1, 1, 1